# DNS Queries to Newly Registered Domains

**DNS queries to domains registered within the last 30 days (requires domain age enrichment).**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **process generating DNS**, **domain reputation**, **first-seen**, **C2 indicators**.


## Investigation Queries

List clients querying newly registered domains; pivot to process/network telemetry.


In [ ]:
// KQL
DnsEvents
| where TimeGenerated > ago(30d)
| where DomainAgeDays <= 30
| where Name contains '<DOMAIN>' or isempty('<DOMAIN>')
| project TimeGenerated, DeviceName, ClientIP, Name, DomainAgeDays
| order by TimeGenerated desc


In [ ]:
# Splunk SPL
index=dns domain_age_days<=30
| stats count values(src_ip) as src by query
| sort - count


In [ ]:
# Elastic (KQL/EQL)
event.category:dns AND threat.enrichment.domain_age_days <= 30


## Containment & Response

Block the domain at DNS/proxy, isolate endpoint if beaconing, and perform malware triage.


In [ ]:
# PowerShell
## Add to Windows hosts file (NOT recommended enterprise-wide; use DNS/proxy controls)
# Add-Content -Path C:\Windows\System32\drivers\etc\hosts -Value "0.0.0.0 <DOMAIN>"


In [ ]:
# Bash
# Example: block domain via DNS sinkhole/proxy (platform-specific)
# Document your control-plane action here.
